# DnD Monsters: Dice and Data
As a Dungeon Master, it is very important to understand the strength of the monsters you pit against your players. Too weak, they are bored, too strong, they die or worse..they don't have fun. The current method known as Challenge Rating, CR, is a numerical system used to determine how difficult an enemey is based on a party of 4 players. Challenge Ratings range from 0 to 30. Unfortunately, this method is very basic and often times does not actually hold true to every encounter. 

One thing that isn't accounted for is action economy. This is the biggest detroyer of players, the strongest weapon in your arsenal. If your players are facing 100 monsters, that's 100 turns. Even if you manage to kill a good chunk of them, the majority will make it through and some of them...with critical hits. Thus is a much more difficult encounter than an equally XP worthwhile monster, with say 2 attacks. 

Wizards of the Coast not only provide a guideline for how much XP you should have per level per day, but they also show you how much a party of 4 at X level can stomach during one encounter. They also provide an XP multiplier that takes multiple monsters into consideration. For example, 10 monsters get a x2.5 XP multiplier, causing their total XP rating to jump up for the encounter, potentially making them deadly. Action Economy rules all. 

CR is unfortunately not a great method for measuring a monster's strength. It uses AC, HP, attack bonus, damage per round and Save DC as a general guideline. It doesn't take into account legendary action, at will spells, special abilities that cause status ailments, or any other boosting abilities.

There are two CRs: Defensive and Offensive, used to calulate the total CR of a monster. Using the chart provided you find the average of the CR indicated by the HP and AC. Offensive does the same thing but uses DPR and Attack Bonus. Then by averaging the two CRs we get our final monster Challenge Rating. As you can see this doesn't take into account any of the strong abilities a monster may have. Similarly, you may have a weak physical monster that uses spells that is vastly lower in CR than it should be. 

WoTC has augmented this system by applying multipliers or increases based on other features, trains, or abilities the monster may have. 

www.dndbeyond.com/monsters has many pages of monster listings. Each listing has a dropdown that has a monster table associated with it. This contains stats, abilities, and other important details. 

Unfortunately, dndbeyond has shut down its ability to scrape through automation detection software. I don't intend to break to ToS, so I will use the SRD from the dandwiki.com page instead. 

The goal of this investigation is to learn more about Monster's abilities in relation to the CR system. To understand if there are corellations in any of the stats, abilities, environments, size, etc. To see if we can classify monsters based on any of these traits. To create a dashboard that pits monsters against each other to compare. Finally, to see if there is a way to better address the CR system and use abilities, traits, features, and spells in a more cohesive manner 


## Libraries for Parsing
First we need to gain access to our monster data sheet. as stated above, dndbeyond.com has a great repository of monster data. This will need to be scrapped from there site. Unfortuntately, each of the monster pages is hidden behind an accordion dropdown and will need to be extracted. This is something I have not yet done, so I am excited to try. We will start out using Requests and BeautifulSoup since I am most comfortable with these.

In [1]:
#Import Libraries for scrapping
from bs4 import BeautifulSoup as bs
import requests as rq
import pandas as pd

## Get Request for Monster Names

In [2]:
#Fetching HTML
url = "https://www.dandwiki.com/wiki/5e_SRD:Monsters"
Request = rq.get(url).text

soup = bs(Request, 'html.parser')

# Collect Names of All Monsters in a List 
Unfortunately, dndwiki is not well crafted, which meant I needed to get creative. There weren't distinguishing classes or names or ids. styles between tables were a bit different, so i used that to gather the information needed.

In [3]:
#Find the main content div and and extract it for processing
#This involves finding the list items that are only housed within the parent table that has a width of 100%.
tables = soup.findAll('table',{'style':"width: 100%;"})
monster_names=[]

for table in tables:
    li_table = table.findAll('li')
    for name in li_table:
          monster_names.append(name.text)

# Clean up data
We need to remove duplicates and non-monsters from the list 

In [4]:
#Remove the non-monster data

#Remove Duplicate monsters if there are any
monster_names = list(set(monster_names))
monster_list=[]
#filter through and replace spaces with dashes to format for urls
for name in monster_names:
    if not(name.strip().isdigit()):
        new_name = name.replace(' ','-')
        monster_list.append(new_name)
    else:
        monster_list.append(name)



# Dictionary of URLs to parse
We will iterate through the monster name, knowing that dandwiki has a uniform site for all monsters pages www.dandwiki.com/wiki/5e_SRD:'MonsterName'.

In [6]:
monster_url=[]
for name in monster_list:
    monster_url.append('https://www.dndbeyond.com/monsters/'+name)


## Iterate through the websites to parse all the data
There are still some things on here that are not monsters (they summon monsters). For example the Deck of Many Things. This will break and analysis or modeling we try to do, so we need to remove them. We can look at all things monsters have in common that these other objects do not. Unfortunately, the DoMT and the figures of power also contain niche "monster" stats for their monsters. We will include these in our table, however Zombies and Dinosaurs do not, since they are just a category of many monsters, all of which are included in the list already. 

In [8]:
from collections import defaultdict

#function to make sure each get request is functioning properly and to parse the url
def Run_Soup_If_Status_Ok(url):
    request =rq.get(url)
    soup = bs(request.text, 'html.parser')
    return soup


monster_dict=defaultdict(list)

#append dictionary with monster name and the soupy information
for name,url in zip(monster_names,monster_url):
    monster_dict[name].append(Run_Soup_If_Status_Ok(url))


# Helper functions for the full Parse

# Create a data frame by parsing the Monster HTML tables
I am going to create an empty dictionary with keys from the extracted column names above. this dictionary will be converted into a pandas dataframe.

# Fill in the monster_dict with records extracted from our HTML


# Create DataFrame from filled monster_dict

In [ ]:
#ensure listlengths are the same
list_length = []

for col in monster_dict:
    list_length.append(len(monster_dict[col]))
print(list_length)

monster_df = pd.DataFrame(monster_dict)

monster_df

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup

url = 'https://www.dndbeyond.com/monsters/mummy-lord'


driver = webdriver.Chrome(executable_path='../env/chromedriver.exe')

driver.get(url)

driver.implicitly_wait(5)

soup = BeautifulSoup(driver.page_source, 'lxml')

stat_block = soup.find('div',{'class':'mon-stat-block'})
Environment = soup.find('footer')




C:\Users\ingal\AppData\Local\Temp\ipykernel_23144\15236720.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../env/chromedriver.exe')


In [52]:

column_names = ['Monster Name','Size','Type', 'Alignment','Traits', 'Damage Resistances', 'Monster Tags', 'Mythic Actions', 'Reactions','Source']
#First set of column names from 'label span'
for headers in stat_block.findAll('span',{'class': lambda e: e.endswith('label') if e else False}):    
    column_names.append(headers.text)
    
for headers in stat_block.findAll('div',{'class': lambda e: e.endswith('heading') if e else False}):    
    column_names.append(headers.text)

for headers in Environment.findAll('p',{'class': lambda e: e.startswith('environment-tags') if e else False}):    
    column_names.append(headers.contents[0].strip())


# Create Empty Dictionary with Keys from the Extracted Column Names

In [166]:
monster_dict = dict.fromkeys(column_names)

#Initialize the monster_dic with each value for all keys to be an empty list
for column in column_names:
    monster_dict[column] = []

monster_dict

{'Monster Name': [],
 'Size': [],
 'Type': [],
 'Alignment': [],
 'Traits': [],
 'Damage Resistances': [],
 'Monster Tags': [],
 'Mythic Actions': [],
 'Reactions': [],
 'Source': [],
 'Armor Class': [],
 'Hit Points': [],
 'Speed': [],
 'Saving Throws': [],
 'Skills': [],
 'Damage Vulnerabilities': [],
 'Damage Immunities': [],
 'Condition Immunities': [],
 'Senses': [],
 'Languages': [],
 'Challenge': [],
 'Proficiency Bonus': [],
 'STR': [],
 'DEX': [],
 'CON': [],
 'INT': [],
 'WIS': [],
 'CHA': [],
 'Actions': [],
 'Legendary Actions': [],
 'Environment:': []}

## Add Values of Mummy Data into our Dictionary

In [167]:
# Monster Name
monster_name = stat_block.find('div', {'class':'mon-stat-block__name'}).text
monster_dict['Monster Name'].append(' '.join(str(monster_name).split())) 


#This next set (Size,Alignment, and Type) will split the single meta text using split() and replace() functions
monster_subinfo = stat_block.find('div', {'class':'mon-stat-block__meta'})
monster_subinfo=monster_subinfo.text

# Size (first word)
monster_size = monster_subinfo.split()[0]
monster_dict['Size'].append(monster_size) 
# Alignment (after comma)
monster_alignment = monster_subinfo.split(', ')[-1]
monster_dict['Alignment'].append(monster_alignment) 
# Type (remaining words). The sublist will remove the above two variables from the text, as well as the loose comma.
#It will also create a list for the type, as sometimes there are sub-types associated with monsters (e.g Titan)
sub_list=(monster_size,monster_alignment, ', ')
monster_type = monster_subinfo
for substring in sub_list:
    monster_type = monster_type.replace(substring,'')
monster_type=monster_type.split()
monster_dict['Type'].append(monster_type) 

#find all attribute metrics
attribute_data = stat_block.findAll('span',{'class':'mon-stat-block__attribute-data-value'})

# Armor Class
monster_ac = ' '.join(str(attribute_data[0].text).split())
monster_dict['Armor Class'].append(monster_ac)
# Hit Points
monster_hp = ' '.join(str(attribute_data[1].text).split())
monster_dict['Hit Points'].append(monster_hp)
# Speed
monster_speed = ' '.join(str(attribute_data[2].text).split())
monster_dict['Speed'].append(monster_speed)


#find all tidbit  metrics
tidbit_label = stat_block.findAll('span', {'class':'mon-stat-block__tidbit-label'})

for label in tidbit_label:    
    '''
    Because the tidbits column shifts based on the monster, we can't index the rows, as they
    are added or deleted based on the monster. So instead, we will write a for loop that loops through 
    the monsters tidbit headings (e.g. Skills, Saving Throws, etc.) and if they exits, it will take
    the sibling data (i.e. it will take the actual data corresponding to each heading) and deposit it into the dictionary.
    Any columns not in the monster data will be left blank for now. Each if statement is labeled with the corresponding tidbit.
    '''
    if label.text == "Saving Throws":
        monster_saving_throw = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Saving Throws'].append(monster_saving_throw)
    elif label.text == "Skills":
        monster_skills = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Skills'].append(monster_skills)
    elif label.text == "Damage Vulnerabilities":    
        monster_damage_vulnerability = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Damage Vulnerabilities'].append(monster_damage_vulnerability)
    elif label.text == "Damage Immunities":
        monster_damage_immunity = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Damage Immunities'].append(monster_damage_immunity)
    elif label.text == 'Condition Immunities':
        monster_condition_immunity = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Condition Immunities'].append(monster_condition_immunity)
    elif label.text == 'Senses':
        monster_senses = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Senses'].append(monster_senses)
    elif label.text == 'Languages':
        monster_languages = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Languages'].append(monster_languages)
    elif label.text == 'Challenge':
        monster_challenge= ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Challenge'].append(monster_challenge)
    elif label.text == 'Proficiency Bonus':
        monster_proficiency = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Proficiency Bonus'].append(monster_proficiency)
    elif label.text == 'Damage Resistences':
        monster_damage_resistence = ' '.join(str(label.find_next_sibling('span').text).split())
        monster_dict['Damage Resistences'].append(monster_damage_resistence)


#find all ability score metrics
ability_scores = stat_block.findAll('span',{'class':'ability-block__score'})
    # STR Score
monster_str = ability_scores[0].text
monster_dict['STR'].append(monster_str)
    # DEX Score
monster_dex = ability_scores[1].text
monster_dict['DEX'].append(monster_dex)
    # CON Score
monster_con = ability_scores[2].text
monster_dict['CON'].append(monster_con)
    # INT Score
monster_int = ability_scores[3].text
monster_dict['INT'].append(monster_int)
    # WIS Score
monster_wis = ability_scores[4].text
monster_dict['WIS'].append(monster_wis)
    # CHA Score
monster_cha = ability_scores[5].text
monster_dict['CHA'].append(monster_cha)    
    
# Traits: because traits doesn't contain any defining HTML or any headings such as Actions or Legendary Actions
# I searched through all the description blocks of the text. If they don't contain the div 'heading' then we print
# This allows us to only print the traits and to place them in a list if need be for later wrangling and analysis. 
             
trait_list = []
description_block = stat_block.findAll('div', {'class':'mon-stat-block__description-block'})
for block in description_block:
     if not block.findAll('div',{'class':'mon-stat-block__description-block-heading'}):
        for p in block.findAll('p'):
            trait_list.append(p.text)

#Remaining descriptions that had headings
description_heading = stat_block.findAll('div', {'class':'mon-stat-block__description-block-heading'})
action_list=[]
for heading in description_heading:    
    '''
    Because the description column shifts based on the monster, we can't index the rows, as they
    are added or deleted based on the monster. So instead, we will write a for loop that loops through 
    the monsters description headings (e.g. Actions, Legendary Actions, etc.) and if they exits, it will take
    the sibling data (i.e. it will take the actual data corresponding to each heading) and deposit it into the dictionary.
    Any columns not in the monster data will be left blank for now. Each if statement is labeled with the corresponding tidbit.
    '''
    action_list=[]
    if heading.text == "Actions":
        monster_actions = heading.find_next_sibling('div')
        for p in monster_actions.findAll('p'):
           action_list.append(p.text.strip())
        monster_dict['Actions'].append(action_list)
    elif heading.text == "Legendary Actions":
        monster_legendary_actions = heading.find_next_sibling('div')
        for p in monster_legendary_actions.findAll('p'):
           action_list.append(p.text.strip())
        monster_dict['Legendary Actions'].append(action_list)
    elif heading.text == "Mythic Actions":
        monster_mythic_actions = heading.find_next_sibling('div')
        for p in monster_mythic_actions.findAll('p'):
           action_list.append(p.text.strip())
        monster_dict['Mythic Actions'].append(action_list)
    elif heading.text == "Reactions":
        monster_reactions = heading.find_next_sibling('div')
        for p in monster_reactions.findAll('p'):
           action_list.append(p.text.strip())
        monster_dict['Reactions'].append(action_list)
         
#These final traits are either referring to the environment it lives in (can be multiple), the sub type its classified as,
# or the source book it came from. all of these or none of these may be represented in the monster sheet.
monster_tags = Environment.findAll('span') 

for tag in Environment.find_all("p"):
       
    if (tag.contents[0].strip()) == "Environment:":
       monster_dict['Environment:'].append(monster_tags[0].text)
    elif (tag.contents[0].strip()) == "Monster Tags:":
        monster_dict['Monster Tags'].append(monster_tags[1].text)
    else:
        monster_dict['Source'].append(tag.contents[0].strip())


In [169]:
description_heading = stat_block.findAll('div', {'class':'mon-stat-block__description-block-heading'})
action_list=[]
for heading in description_heading:    
    '''
    Because the description column shifts based on the monster, we can't index the rows, as they
    are added or deleted based on the monster. So instead, we will write a for loop that loops through 
    the monsters description headings (e.g. Actions, Legendary Actions, etc.) and if they exits, it will take
    the sibling data (i.e. it will take the actual data corresponding to each heading) and deposit it into the dictionary.
    Any columns not in the monster data will be left blank for now. Each if statement is labeled with the corresponding tidbit.
    '''
    action_list=[]
    if heading.text == "Legendary Actions":
        monster_legendary_actions = heading.find_next_sibling('div')
        for p in monster_legendary_actions.findAll('p'):
           action_list.append(p.text.strip())
action_list

["The mummy lord can take 3 legendary actions, choosing from the options below. Only one legendary action option can be used at a time and only at the end of another creature's turn. The mummy lord regains spent legendary actions at the start of its turn.",
 'Attack. The mummy lord makes one attack with its rotting fist or uses its Dreadful Glare.',
 "Blinding Dust. Blinding dust and sand swirls magically around the mummy lord. Each creature within 5 feet of the mummy lord must succeed on a DC 16 Constitution saving throw or be blinded until the end of the creature's next turn.",
 "Blasphemous Word (Costs 2 Actions). The mummy lord utters a blasphemous word. Each non-undead creature within 10 feet of the mummy lord that can hear the magical utterance must succeed on a DC 16 Constitution saving throw or be stunned until the end of the mummy lord's next turn.",
 "Channel Negative Energy (Costs 2 Actions). The mummy lord magically unleashes negative energy. Creatures within 60 feet of the

In [164]:
#ensure listlengths are the same
import pandas as pd


monster_dict = dict([ (k,pd.Series(v, dtype=int)) for k,v in monster_dict.items()])
monster_dict
#list_length = []

#for col in monster_dict:
#    list_length.append(len(monster_dict[col]))
#print(list_length)
#
monster_df = pd.DataFrame(monster_dict)
#
monster_df

ValueError: invalid literal for int() with base 10: 'Mummy Lord'

In [150]:
monster_df.to_csv('../data/raw/MummyTest.csv')